<a href="https://colab.research.google.com/github/ArashDehghanyan/ml-practicres/blob/main/Save_and_load_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup
##Installs and imports

In [1]:
! pip install pyyaml h5py   # Required to save models in HDF5 format

In [2]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.version.VERSION)

2.8.0


In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.
train_labels = train_labels[:1000]

test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0
test_labels = test_labels[:1000]

11501568/11490434 [==============================] - 0s 0us/step


#Define a model
##Start by a simple Sequential model

In [4]:
def create_model():
    # Define model
    model = tf.keras.models.Sequential([
        layers.Dense(512, activation='relu', input_shape=(784,)),
        layers.Dropout(0.2),
        layers.Dense(10)
    ])
    # Compile model
    model.compile(
        optimizer=keras.optimizers.Adam(0.001),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy()]
    )

    return model

# Create a basic model instance
basic_model = create_model()

# Display the model architecture
basic_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


#Save checkpoints during training

In [5]:
checkpoint_path = "training_1/checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weight
checkpoint_callback = keras.callbacks.ModelCheckpoint(checkpoint_path, verbose=1, save_weights_only=True)

# Train model with new callback
basic_model.fit(
    train_images,
    train_labels,
    epochs=10,
    validation_data=(test_images, test_labels),
    # Pass callback to training
    callbacks=[checkpoint_callback]
)

Epoch 1/10
28/32 [=========================>....] - ETA: 0s - loss: 1.2099 - sparse_categorical_accuracy: 0.6674
Epoch 1: saving model to training_1/checkpoint.ckpt
32/32 [==============================] - 1s 12ms/step - loss: 1.1300 - sparse_categorical_accuracy: 0.6920 - val_loss: 0.6858 - val_sparse_categorical_accuracy: 0.7960
Epoch 2/10
29/32 [==========================>...] - ETA: 0s - loss: 0.4247 - sparse_categorical_accuracy: 0.8772
Epoch 2: saving model to training_1/checkpoint.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.4153 - sparse_categorical_accuracy: 0.8780 - val_loss: 0.5201 - val_sparse_categorical_accuracy: 0.8420
Epoch 3/10
28/32 [=========================>....] - ETA: 0s - loss: 0.2776 - sparse_categorical_accuracy: 0.9297
Epoch 3: saving model to training_1/checkpoint.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.2835 - sparse_categorical_accuracy: 0.9270 - val_loss: 0.4718 - val_sparse_categorical_accuracy: 0.8500
Epo

In [6]:
os.listdir(checkpoint_dir)

['checkpoint', 'checkpoint.ckpt.data-00000-of-00001', 'checkpoint.ckpt.index']

##Create an untrained model and evalute it on the test set

In [7]:
untrained_model = create_model()

# Evaluate the model
loss, acc = untrained_model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3820 - sparse_categorical_accuracy: 0.0930 - 156ms/epoch - 5ms/step
Untrained model accuracy:  9.30%


###Load weights from checkpoint and re-evaluate the model

In [8]:
# Load weights
untrained_model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = untrained_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4038 - sparse_categorical_accuracy: 0.8720 - 68ms/epoch - 2ms/step
Restored model accuracy: 87.20%


##Checkpoint options

In [9]:
# Include the epoch in the filename
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback to save model's weight every 5 epochs
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size
)

# Create a new model instance
new_model = create_model()

# Save the weights using (checkpoint_path) format
new_model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with new callback
new_model.fit(
    train_images,
    train_labels,
    epochs=50,
    verbose=1, 
    callbacks=[cp_callback],
    batch_size=batch_size,
    validation_data=(test_images, test_labels)
)

Epoch 1/50
32/32 [==============================] - 1s 10ms/step - loss: 1.1695 - sparse_categorical_accuracy: 0.6760 - val_loss: 0.7257 - val_sparse_categorical_accuracy: 0.7830
Epoch 2/50
32/32 [==============================] - 0s 6ms/step - loss: 0.4272 - sparse_categorical_accuracy: 0.8740 - val_loss: 0.5582 - val_sparse_categorical_accuracy: 0.8240
Epoch 3/50
32/32 [==============================] - 0s 7ms/step - loss: 0.2917 - sparse_categorical_accuracy: 0.9340 - val_loss: 0.4801 - val_sparse_categorical_accuracy: 0.8530
Epoch 4/50
32/32 [==============================] - 0s 6ms/step - loss: 0.2193 - sparse_categorical_accuracy: 0.9450 - val_loss: 0.4263 - val_sparse_categorical_accuracy: 0.8680
Epoch 5/50
27/32 [========================>.....] - ETA: 0s - loss: 0.1576 - sparse_categorical_accuracy: 0.9664
Epoch 5: saving model to training_2/cp-0005.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.1628 - sparse_categorical_accuracy: 0.9640 - val_loss: 0.4214 

In [10]:
os.listdir(checkpoint_dir)

['cp-0035.ckpt.index',
 'cp-0015.ckpt.index',
 'checkpoint',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index']

In [11]:
# Select the latest checkpoint
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [12]:
# Create a new model instance
new_untrained_model = create_model()

# Load previously saved weights
new_untrained_model.load_weights(latest)

# Re-evaluate the model
loss, acc = new_untrained_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model accuracy: {:5.2f}%".format(acc * 100))


32/32 - 0s - loss: 0.4780 - sparse_categorical_accuracy: 0.8760 - 164ms/epoch - 5ms/step
Restored model accuracy: 87.60%


#Manually save models

In [13]:
# Save the weights
basic_model.save_weights('./checkpoints/my-checkpoints')

#  Create model
model = create_model()

# Restore model
model.load_weights("./checkpoints/my-checkpoints")

# Evaluate model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4038 - sparse_categorical_accuracy: 0.8720 - 161ms/epoch - 5ms/step
Restored model, accuracy: 87.20%


##SavedModel format

In [14]:
# Create and train a new model instance
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save the rntire model as a savedmodel
! mkdir -p saved_model
model.save("saved_model/my_model")


Epoch 1/5
32/32 [==============================] - 1s 6ms/step - loss: 1.1462 - sparse_categorical_accuracy: 0.6850
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4189 - sparse_categorical_accuracy: 0.8910
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2905 - sparse_categorical_accuracy: 0.9280
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2072 - sparse_categorical_accuracy: 0.9460
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1510 - sparse_categorical_accuracy: 0.9670
INFO:tensorflow:Assets written to: saved_model/my_model/assets


##Inspect the saved model directory

In [15]:
# contains my_model directory
! ls saved_model

# Contains assets directory, keras_metadata.pb, saved_model.pb and variables folder.
! ls saved_model/my_model/

my_model
assets	keras_metadata.pb  saved_model.pb  variables


##Reload a fresh keras model

In [16]:
new_model = keras.models.load_model("saved_model/my_model")

# Check its architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))


print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4469 - sparse_categorical_accuracy: 0.8490 - 153ms/epoch - 5ms/step
Restored model, accuracy: 84.90%
(1000, 10)


##HDF5 format

In [19]:
# Create a new model
model = create_model()

# Train model
model.fit(train_images, train_labels, epochs=5, verbose=1)

# Save the whole model to a HDF5 file.
# the .h5 extension indicates that file is in HDF5 format.
model.save("my_model.h5")

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1788 - sparse_categorical_accuracy: 0.6690
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4396 - sparse_categorical_accuracy: 0.8720
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2862 - sparse_categorical_accuracy: 0.9270
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2184 - sparse_categorical_accuracy: 0.9470
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1604 - sparse_categorical_accuracy: 0.9660


In [20]:
# Recreate the exact same model, including its weights and optimizer
new_model = keras.models.load_model("my_model.h5")

# Show model structure
new_model.summary()

# Check its accuracy
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model accuracy: {:5.2f}%".format(100 * acc))



Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
32/32 - 0s - loss: 0.4369 - sparse_categorical_accuracy: 0.8680 - 157ms/epoch - 5ms/step
Restored model accuracy: 86.80%
